In [39]:
from bs4 import BeautifulSoup

In [40]:
import requests
from urllib.parse import urljoin, urlparse
import os
import re

In [ ]:
base_url="https://www.cfsph.iastate.edu/"

In [ ]:
r=requests.get("https://www.cfsph.iastate.edu/diseaseinfo/disease-images-list/")
soup = BeautifulSoup(r.content, "html.parser")

In [43]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="tr"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="tr"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="tr"> <![endif]-->
<!--[if IE 9]>    <html class="no-js lt-ie10" lang="tr"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="tr">
 <!--<![endif]-->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script>
   if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}
  </script>
  <script>
   (()=>{class Rocket

In [ ]:
images=soup.find_all("div",attrs={"class":"even-row"})

In [45]:
images

[<div class="thumbnail-container" style="padding-bottom:78.947%"><img alt="Oğlaklarda Cryptosporidiosis" class="wp-post-image" data-lazy-src="https://vetrehberi.com/wp-content/uploads/2017/11/Oğlaklarda-Cryptosporidiosis-475x375.jpg" decoding="async" fetchpriority="high" height="375" src="data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20475%20375'%3E%3C/svg%3E" width="475"><noscript><img alt="Oğlaklarda Cryptosporidiosis" class="wp-post-image" decoding="async" fetchpriority="high" height="375" src="https://vetrehberi.com/wp-content/uploads/2017/11/Oğlaklarda-Cryptosporidiosis-475x375.jpg" width="475"/></noscript></img></div>]

In [ ]:
output_folder = "downloaded_images"
os.makedirs(output_folder, exist_ok=True)  # Klasör yoksa oluştur

for image in images:
    # <a> etiketlerini bul ve 'href' niteliğini al
    linkler = image.find_all("a", attrs={"href": True})
    for link in linkler:
        relative_href = link.get("href")  # Göreli URL
        href = urljoin(base_url, relative_href)  # Tam URL'yi oluştur
        print(f"Link: {href}")

        try:
            # Linkin HTML içeriğini al
            response = requests.get(href)
            response.raise_for_status()  # Hata durumunda exception fırlat
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Sayfadaki tüm <img> etiketlerini bul
            img_tags = soup.find_all("img")
            for img in img_tags:
                img_src = img.get("src")
                if img_src:  # Eğer 'src' varsa
                    img_url = urljoin(base_url, img_src)  # Tam URL'yi oluştur
                    print(f"İndiriliyor: {img_url}")

                    try:
                        # Görseli indir
                        img_data = requests.get(img_url).content
                        
                        # Dosya adını oluştur
                        img_name = os.path.basename(img_url)
                        file_path = os.path.join(output_folder, img_name)

                        # Görseli kaydet
                        with open(file_path, "wb") as f:
                            f.write(img_data)
                        print(f"Görsel kaydedildi: {file_path}")
                    except requests.exceptions.RequestException as e:
                        print(f"Görsel indirilemedi: {img_url}, Hata: {e}")
        except requests.exceptions.RequestException as e:
            print(f"Linke gidilemedi: {href}, Hata: {e}")

İndiriliyor: data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E
Görsel indirilemedi: data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E, Hata: No connection adapters were found for "data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E"
İndiriliyor: https://vetrehberi.com/wp-content/uploads/2017/11/logo.png
Görsel kaydedildi: downloaded_images\logo.png
İndiriliyor: data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E
Görsel indirilemedi: data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E, Hata: No connection adapters were found for "data:image/svg+xml,%3Csvg%20xmlns='http://www.w3.org/2000/svg'%20viewBox='0%200%20230%2041'%3E%3C/svg%3E"
İndiriliyor: https://vetrehberi.com/wp-content/uploads/2017/11/logo.png
Görsel kaydedildi: